Exercise for Unit 6

BSCS 3A-AI

Luis Rafael L. Ajoc

John Felmer B. Robles

In [5]:
# !pip install -U 'tensorflow[and-cuda]'
import sentence_transformers

In [6]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [7]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
# Dataset taken from https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset

train_dir = "/kaggle/input/vegetable-image-dataset/vegetable_image_dataset/train"
test_dir = "/kaggle/input/vegetable-image-dataset/vegetable_image_dataset/test"
val_dir = "/kaggle/input/vegetable-image-dataset/vegetable_image_dataset/validation"

In [8]:
class_names = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']


In [9]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [10]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 12000 images belonging to 15 classes.
Found 600 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [11]:
# class_names = tf.keras.utils.image_dataset_from_directory(train_dir).class_names
# print(class_names)

In [12]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(15, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])

I0000 00:00:1763548604.472089      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763548604.472905      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1763548608.566602     131 service.cc:148] XLA service 0x7de77400ddd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763548608.567975     131 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763548608.568006     131 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763548609.074509     131 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/375 ━━━━━━━━━━━━━━━━━━━━ 47:37 8s/step - accuracy: 0.0312 - loss: 1.1562

I0000 00:00:1763548614.080385     131 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


375/375 ━━━━━━━━━━━━━━━━━━━━ 103s 254ms/step - accuracy: 0.1622 - loss: 0.4382 - val_accuracy: 0.4900 - val_loss: 0.2200
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 64s 172ms/step - accuracy: 0.3576 - loss: 0.2283 - val_accuracy: 0.4783 - val_loss: 0.2020
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 64s 172ms/step - accuracy: 0.4125 - loss: 0.2109 - val_accuracy: 0.4833 - val_loss: 0.1931
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 62s 166ms/step - accuracy: 0.4466 - loss: 0.2023 - val_accuracy: 0.4850 - val_loss: 0.1917
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 62s 166ms/step - accuracy: 0.4554 - loss: 0.1984 - val_accuracy: 0.4950 - val_loss: 0.1842
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 63s 167ms/step - accuracy: 0.4687 - loss: 0.1943 - val_accuracy: 0.5067 - val_loss: 0.1924
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 61s 162ms/step - accuracy: 0.4787 - loss: 0.1945 - val_accuracy: 0.5400 - val_loss: 0.1804
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 61s 164ms/step - accuracy: 0.4854 - loss: 0.1922 - va

In [15]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 143ms/step - accuracy: 0.4685 - loss: 0.1929
Test Accuracy: 0.5866666436195374


In [16]:
# SAVE THE MODEL
model.save('/kaggle/working/exercise_6_custom_ajoc.h5', save_format='h5')

In [17]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='/kaggle/working/exercise_6_custom_ajoc.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)[0]
    pred_index = np.argmax(predictions)
    pred = predictions[pred_index]
    label = class_names[pred_index]
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [18]:
# Example usage:
predict_image("/kaggle/input/vegetable-image-dataset/vegetable_image_dataset/run/run_papaya.jpg")
predict_image("/kaggle/input/vegetable-image-dataset/vegetable_image_dataset/run/run_tomato.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 623ms/step


Prediction: Tomato (confidence: 0.40)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
Prediction: Capsicum (confidence: 0.71)
